# Problem 1

(1) Consider the cubic polynomial $z^3-1$.  The roots are $1, e^{2 \pi i/3}$ and $e^{-2 \pi i /3}$. Write a python function that takes as input four floting point numbers $x_0, x_1, y_0, y_1$ and an integer $n$, and plots (on the screen) which root Newton's method converges to, when one uses an initial guess in a grid of point in the rectangle $[x_0, x_1] \times [y_0, y_1]$, using resolution $n$ for the grid.  Use colours to indicate which root Newton's method finds.  Include a plot for the rectangle $[-2,2] \times [-2,2]$ with $n=400$.  I have included a symbolic version of Newton's method that allows you to use complex numbers as input, it is a slight modification of the algorithm from the second week.

In [8]:
## Newton's method that allows for complex number input.
import sympy
import mpmath

# Define some things for mpmath.
mpmath.dps = 20
mpmath.prec = mpmath.dps * 3.34
mpmath.pretty = True

# Define the symbolic functions.
sym_x = sympy.Symbol("x")
sym_func = (sym_x ** 3) - 1

def newtons_method(f):
    return sym_x - f / sympy.diff(f,sym_x)

newton_iterator = sympy.lambdify(sym_x, newtons_method(sym_func), "mpmath")

# Here is an example of how Newton's method for z^3 - 1 converges to two different
z = mpmath.mpc(1.0, 0.5) ## mpc is the mpmath arbitrary-precision complex number floating-point type.
print("Initial condition: ", z)

for i in range(0,5):
    z = newton_iterator(z)
    print(z)

print("Evaluating ", sym_func, " at ", z, " gives: ", sym_func.evalf(subs = {sym_x : z}))


# Second Example
z = mpmath.mpc(-1,0.5)
print("Initial condition: ", z)

for i in range(0,5):
    z = newton_iterator(z)
    print(z)

print("Evaluating ", sym_func, " at ", z, " gives: ", sym_func.evalf(subs = {sym_x : z}))

Initial condition:  (1.0 + 0.5j)
(0.826666666666667 + 0.12j)
(1.0091012932686 - 0.0558270112377784j)
(0.99709137869203 - 0.00121867816077464j)
(1.00000699042881 + 7.12833910918834e-6j)
(0.999999999998054 + 9.96593837084014e-11j)
Evaluating  x**3 - 1  at  (0.999999999998054 + 9.96593837084014e-11j)  gives:  -5.8386629162883e-12 + 2.9897815112404e-10*I
Initial condition:  (-1.0 + 0.5j)
(-0.506666666666667 + 0.546666666666667j)
(-0.383281777777778 + 0.962716444444444j)
(-0.481017397612345 + 0.855182678305867j)
(-0.500490032992377 + 0.866013502843391j)
(-0.500000109982395 + 0.866025190244472j)
Evaluating  x**3 - 1  at  (-0.500000109982395 + 0.866025190244472j)  gives:  -3.89819586060623e-7 + 6.06052437207287e-7*I


# Problem 2

(2) In class we have seen implementations of the mid-point rule and Simpson's 3/8-method to approximate integrals. One of the key features of these methods (especially useful when your function comes from measured data) is that the technique uses as its input **only** the values of $f(x)$ at a finite number of points.  Sometimes one has access to **more** information about $f$ than simply its values.  In this problem, we ask you to write a script to approximate integrals numerically, but where one has a **symbolic** expression for $f$, and therefore using sympy, one also has access to $f'$. 

**Fact** Given an interval $[a,b]$ and numbers $f_a, f'_a, f_b, f'_b$ there exists one (and only one) cubic polynomial $p(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3$ such that $$p(a) = f_a, \ \ p'(a) = f'_a, \ \ p(b) = f_b, \ \ p'(b) = f'_b$$

Moreover, one can solve for $c_0,c_1,c_2,c_3$ fairly easily.  The above equations are:

$$c_0 + c_1 a + c_2 a^2 + c_3 a^3 = f_0 \ [eq.1]\ \ c_0 + c_1 b + c_2 b^2 + c_3 b^3 = f_1 \ [eq.2] $$
$$c_1 + 2c_2a + 3c_3a^2 = f'_0 \ [eq.3] \ \ c_1 + 2c_2b+3c_3b^2 = f'_1 \ [eq.4] $$

a little manipulation and one gets to the expression:

$$\pmatrix{ c_2 \\ c_3} = \frac{1}{(a-b)^3} \cdot \pmatrix{ 3(b+a) & -2(b^2+ba+a^2) \\ -2 & b+a} \cdot \pmatrix{ f_0 - f_1 + bf_1' - af_0' \\ f_1' -f_0'}$$

the middle expression is a $2 \times 2$ matrix. From the above one can solve for $c_0$ and $c_1$ by back-substituting into [eq.1] and [eq.3] (or [eq.2] and [eq.4], your choice). 

Compare the accuracy of this method to the mid-point and Simpson 3/8-method, for the computation of the integral $\int_0^\pi \sin(x) dx = 2$. Try the number of intervals $n=10, 100, 1000$ for all three methods. 

Aside from accuracy, what other considerations would cause you to prefer one method over the other two?

In [23]:
import sympy
import mpmath
import numpy

def midpoint_integral(f, I, k): 
    sum = 0
    deltax = (I[1]-I[0])/float(k)
    for i in range (0, k):
        x = ((i/float(k))*I[1]) + ((1.0-(i/float(k)))*I[0]) + deltax/2
        sum = sum + f(x)*deltax
    return sum


def three_eights_integral(f, I, k): 
    sum = 0
    deltax = (I[1]-I[0])/k
    for i in range (0, k):
        xa = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) 
        xb = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) + deltax/3 
        xc = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) + (2*deltax)/3
        xd = ((i/float(k))*I[1]) + ((1-i/float(k))*I[0]) + deltax
        sum = sum + ( f(xa) + 3*f(xb) + 3*f(xc) + f(xd) )*deltax/8
    return sum

subdivisions = 20

print("Midpoint with 10 subdivisions: ",
      midpoint_integral(numpy.sin, [0.0, numpy.pi], subdivisions),
      "Error ",
      2.0 - midpoint_integral(numpy.sin, [0.0, numpy.pi], subdivisions))
print("Three eights integral: ",
      three_eights_integral(numpy.sin, [0.0, numpy.pi], int(subdivisions / 3)),
      "Error ",
      2.0 - three_eights_integral(numpy.sin, [0.0, numpy.pi], int(subdivisions / 3)))


Midpoint with 10 subdivisions:  2.00205764829 Error  -0.00205764828542
Three eights integral:  2.00002336737 Error  -2.33673671697e-05
